In [1]:
# The import statement will vary depending on your LLM and vector database. This is an example for OpenAI + ChromaDB

from vanna.openai.openai_chat import OpenAI_Chat
from vanna.chromadb.chromadb_vector import ChromaDB_VectorStore

class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)


In [6]:
from Setting import openAIKey
from Setting import databaseSetting
vn = MyVanna(config={'api_key': openAIKey.Key, 'model': 'gpt-3.5-turbo'})



In [7]:
from sqlalchemy import create_engine
username=databaseSetting.username
password=databaseSetting.password
host=databaseSetting.host
port=databaseSetting.port
database='testdb'
# 创建连接引擎
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')


In [9]:
import pandas as pd

file_path='/Users/tanghao/NUS/Internship/EverpixAnalysis/Data/External Metrics/Old Flurry Metrics.csv'
file='Old_Flurry_Metrics'
df = pd.read_csv(file_path)
df.to_sql(file, engine, if_exists='replace', index=False)

365

In [11]:
from Setting import databaseSetting

vn.connect_to_postgres(
    host=databaseSetting.host,
    dbname='testdb',
    user=databaseSetting.username,
    port=databaseSetting.port,
    password=databaseSetting.password
)

In [13]:
# The information schema query may need some tweaking depending on your database. This is a good starting point.
sql="""
SELECT *
FROM information_schema.columns
WHERE table_name = 'Old_Flurry_Metrics';
"""
df_information_schema = vn.run_sql(sql)

df_information_schema

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,testdb,public,Old_Flurry_Metrics,ActiveUsers,1,None,YES,bigint,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,testdb,public,Old_Flurry_Metrics,AvgPageViewsPerSession,2,None,YES,bigint,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,testdb,public,Old_Flurry_Metrics,AvgSessionLength,3,None,YES,bigint,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
3,testdb,public,Old_Flurry_Metrics,MedianSessionLength,4,None,YES,bigint,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
4,testdb,public,Old_Flurry_Metrics,NewUsers,5,None,YES,bigint,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
5,testdb,public,Old_Flurry_Metrics,PageViews,6,None,YES,bigint,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
6,testdb,public,Old_Flurry_Metrics,RetainedUsers,7,None,YES,bigint,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
7,testdb,public,Old_Flurry_Metrics,Sessions,8,None,YES,bigint,None,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
8,testdb,public,Old_Flurry_Metrics,date,9,None,YES,text,None,1.073742e+09,...,NO,None,None,None,None,None,NO,NEVER,None,YES


In [14]:
# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
plan

# If you like the plan, then uncomment this and run it to train
# vn.train(plan=plan)

Train on Information Schema: testdb.public Old_Flurry_Metrics

In [16]:
plan.get_summary()

['Train on Information Schema: testdb.public Old_Flurry_Metrics']

In [ ]:
# The following are methods for adding training data. Make sure you modify the examples to match your database.

# DDL statements are powerful because they specify table names, colume names, types, and potentially relationships
vn.train(ddl="""
    CREATE TABLE IF NOT EXISTS my-table (
        id INT PRIMARY KEY,
        name VARCHAR(100),
        age INT
    )
""")

# Sometimes you may want to add documentation about your business terminology or definitions.
vn.train(documentation="Our business defines OTIF score as the percentage of orders that are delivered on time and in full")

# You can also add SQL queries to your training data. This is useful if you have some queries already laying around. You can just copy and paste those from your editor to begin generating new SQL.
vn.train(sql="SELECT * FROM my-table WHERE name = 'John Doe'")

In [17]:
# At any time you can inspect what training data the package is able to reference
training_data = vn.get_training_data()
training_data

,id,question,content,training_data_type


In [18]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on
